# Mask Classification using PyTorch

In [ ]:
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
# loading dataset
#data = pd.read_csv('G:/AI Project/mask-classification-pytorch/dataset.csv')
#data.head()

data_path = "G:/AI Project/mask-classification-pytorch/dataset.csv"
img_path = "G:\\AI Project\\mask-classification-pytorch\\dataset\\"

# Loading Data

In [ ]:
import os
from torch.utils.data import Dataset
from PIL import Image

class MaskImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [ ]:
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                ])

full_dataset = MaskImageDataset(data_path, img_path, transform=transform)

In [ ]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [ ]:
train_data = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_data = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Creating Training Model

In [ ]:
from torch.nn import Conv2d, Linear, MaxPool2d, Module, BatchNorm2d
from torch.nn import functional as F

class MaskNetV2(Module):
    
    def __init__(self):
        ''' Initializing the model'''
        super(MaskNetV2, self).__init__()
        
        self.conv1_1 = Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.bn_1 = BatchNorm2d(32)
        
        self.conv2_1 = Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0)
        self.bn_2 = BatchNorm2d(64)
        
        self.conv3_1 = Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=0)
        self.bn_3 = BatchNorm2d(128)
        
        self.conv4_1 = Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=0)
        self.bn_4 = BatchNorm2d(256)
        
        self.maxpool = MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fc1 = Linear(36864, 1024)
        self.fc2 = Linear(1024, 512)
        self.fc3 = Linear(512, 5)
        
        
    def forward(self, x):
        
        x = F.relu(self.bn_1(self.conv1_1(x)), inplace=True)
        x = self.maxpool(x)
        
        x = F.relu(self.bn_2(self.conv2_1(x)), inplace=True)
        x = self.maxpool(x)
        
        x = F.relu(self.bn_3(self.conv3_1(x)), inplace=True)
        x = self.maxpool(x)
        
        x = F.relu(self.bn_4(self.conv4_1(x)), inplace=True)
        x = self.maxpool(x)
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x), inplace=True)
        x = F.dropout(x, 0.3)
        
        x = F.relu(self.fc2(x), inplace=True)
        x = F.dropout(x, 0.3)
        
        x = self.fc3(x)
        return x

In [ ]:
model = MaskNetV2()
print(model)

# Training Model

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
print(device)

In [ ]:
def trainModel(model, train_loader, device):
    learning_rate = 0.001
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),lr = learning_rate)
    
    train_losses = []
    acc_list = []
    epochs = 45
    
    for i in range(1, epochs+1):
        start = time.time()
        
        running_loss = 0.0
        total = 0
        correct = 0
        
        for j, data in enumerate(train_loader, 0):
            
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            total += labels.size(0)
            _, predicted = torch.max(outputs.data, 1) 
            correct += (predicted == labels).sum().item() 
        
            
        
        train_loss = running_loss/len(train_loader.sampler)
        train_losses.append(train_loss)
        accuracy = (correct / total) * 100
        acc_list.append(accuracy)
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tAccuracy: {:.6f}'.format(
        i, train_loss, accuracy))
        elapsed = time.time() - start
        print("Elapsed time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed % 1)[2:])[:11], time.gmtime(elapsed)))
    
    print("Finished Training")
    torch.save(model.state_dict(), "G:/AI Project/mask-classification-pytorch/saved_models/masknetv2_4.pt")
    torch.save(model, "G:/AI Project/mask-classification-pytorch/saved_models/masknetv2_4_full.pt")

In [ ]:
trainModel(model, train_data, device)

# Model Evalution

In [ ]:
import torch
from torch.utils.data import  DataLoader,random_split
from torchvision import datasets,transforms
import torch.nn as nn

import numpy as np

import matplotlib.pyplot as plt


data_path = "G:/AI Project/mask-classification-pytorch/dataset.csv"
img_path = "G:\\AI Project\\mask-classification-pytorch\\dataset\\"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu") 
#device = torch.device('cpu')

transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                ])

full_dataset = MaskImageDataset(data_path, img_path, transform=transform)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])
#train_data = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_data = DataLoader(test_dataset, batch_size=32, shuffle=True)

the_model = MaskNetV2()
the_model = the_model.to(device)
path = "G:/AI Project/mask-classification-pytorch/saved_models/masknetv2_3_full.pt"
the_model=torch.load(path)
#the_model=torch.load("G:/AI Project/mask-classification-pytorch/saved_models/masknetv2_3_full.pt",torch.device('cpu'))

## Test Accuracy

In [ ]:
from torch.nn.functional import cross_entropy
def test_model(model,testing_data,DEVICE):
      
    testing_loss = 0
    correct_prediction = 0 
    data_size = 0
    prediction1=[]
    for images, labels in testing_data:
            images = images.to(device)
            labels = labels.to(device)          
            data_size += len(images)
            prediction = model(images)
            
            prediction1.append(prediction)
            
            testing_loss += cross_entropy(prediction, labels).item()
            correct_prediction += (prediction.argmax(dim=1) == labels).sum().item()


    accuracy = correct_prediction/data_size
    testing_loss = testing_loss/data_size

    print('\nTesting:')
    print(f"Correct prediction: {correct_prediction}/{data_size} and accuracy: {accuracy} and loss: {testing_loss}")

In [ ]:
torch.cuda.empty_cache()
test_model(the_model,test_data,device)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
def get_labels_N_prediction(model,loader,DEVICE):
    all_labels = []
    all_prediction = []

    for batch in loader:
        images, labels = batch
        images = images.to(DEVICE)

        prediction = model(images).to(torch.device("cpu")).argmax(dim=1).detach().numpy()
        labels = labels.to(torch.device("cpu")).detach().numpy()

        all_prediction = np.append(all_prediction,prediction)
        all_labels = np.append(all_labels,labels)

    return [all_labels,all_prediction]
# Evaluation
with torch.no_grad():
    labels_N_prediction = get_labels_N_prediction(the_model, test_data, device)

    
print(classification_report(labels_N_prediction[0], labels_N_prediction[1]))
conf_matrix = confusion_matrix(labels_N_prediction[0], labels_N_prediction[1])
print(conf_matrix)

## Accuracy Score

In [ ]:
#%%
import pandas as pd
#ACCURACY SCORE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib
#y_pred = the_model.predict(test_dataset)
y_pred =labels_N_prediction[1]
y_test =labels_N_prediction[0]
print("Accuracy : ",accuracy_score(y_test, y_pred))

## Precision, Recal, FScore, Support

In [ ]:
#PRECISION , RECALL,FSCORE,SUPPORT
from sklearn.metrics import precision_recall_fscore_support
d=()
d=precision_recall_fscore_support(labels_N_prediction[0], y_pred)
prec = d[0].tolist()
recall = d[1].tolist()
fscore = d[2].tolist()
support = d[3].tolist()
#precision_recall_fscore_support(y_test, y_pred,average="macro")
print("prec --",prec)
print("recall --",recall)
print("fscore --",fscore)
print("support --",support)

## Classification Report

In [ ]:
#CLASSIFICATION REPORT - PRECISION , RECALL,FSCORE,SUPPORT
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

#PLOTTING CLASSIFICATION REPORT
import seaborn as sns
from pylab import savefig
h= classification_report(y_test, y_pred , output_dict=True)
svm =sns.heatmap(pd.DataFrame(h).iloc[:-1, :].T, annot=True)
figure = svm.get_figure()

figure.savefig('Classification report.png')

#CONFUSION MATRIX & PLOTTING IT
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
confusion_matrix = confusion_matrix(y_test, y_pred).tolist()
cm = sns.heatmap(confusion_matrix, annot=True, fmt='d')


cm.plot()
matplotlib.pyplot.show()
matplotlib.pyplot.savefig('confusion metrics.png')

#PLOTTING PRECISION AND RECALL GRAPH
from sklearn.metrics import PrecisionRecallDisplay

disp = PrecisionRecallDisplay(precision=precision_recall_fscore_support(y_test, y_pred)[0],recall=precision_recall_fscore_support(y_test, y_pred)[1])
disp.plot()
matplotlib.pyplot.show()
matplotlib.pyplot.savefig('Precision vs Recall.png')

# Testing trained model on new Image

## Loading saved trained model

In [ ]:
model = torch.load("G:/AI Project/mask-classification-pytorch/saved_models/masknetv2_3_full.pt")

In [ ]:
import torch

from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt


def predictImage(model, imagePath, device, labels={0:'cloth', 1:'N95', 2:'N95 with valve', 3:'No Mask', 4:'Surgical'}):
    
    
    transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                ])
    image = Image.open(imagePath).convert('RGB')
    imageD = Image.open(imagePath).convert('RGB')
    
    image = transform(image)
    
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        output = model(image.unsqueeze(0))
    pred = list(output.argmax(dim=1).cpu().numpy())
    #print(pred)
    
    plt.imshow(imageD)    
    print("Prediction : " + labels[pred[0]])

In [ ]:
predictImage(model, "G:/AI Project/mask-classification-pytorch/try1.jpg", "cuda")

In [ ]:
print(model)